In [19]:
import pandas as pd
import re

df = pd.read_csv('issued-building-permits.csv', on_bad_lines='skip', sep=';', engine='python')

def extract_postal_code(address):
    # Regular expression for the pattern of a postal code
    pattern = r'.*\s\d[A-Z]\d$'
    if re.match(pattern, str(address)):
        return address[-7:]  # Return the last 7 characters if pattern matches
    else:
        return 'N/A'  # Return 'N/A' if pattern does not match

df['PostalCode'] = df['Address'].apply(extract_postal_code)


df = df.drop(columns=['PermitNumber', 'PermitNumberCreatedDate','PermitElapsedDays', 'ProjectValue','ProjectDescription',
                      'PermitCategory', 'Applicant', 'ApplicantAddress','PropertyUse', 'SpecificUseCategory', 'BuildingContractor',
                      'BuildingContractorAddress','IssueDate','GeoLocalArea', 'Geom', 'YearMonth', 'Address'])


filtered_df = df[~df['TypeOfWork'].isin(['Demolition / Deconstruction', 'Salvage and Abatement', 'Outdoor Uses (No Buildings Proposed)'])]

filtered_df['Cordinates'] = filtered_df['geo_point_2d']
filtered_df = filtered_df.drop(columns=['geo_point_2d'])
filtered_df = filtered_df.dropna(subset=['Cordinates'])
filtered_df = filtered_df[filtered_df['Cordinates'] != '']

print(filtered_df.head())

              TypeOfWork  IssueYear PostalCode                Cordinates
2           New Building       2019    V5V 2Y8  49.2416496, -123.0729656
3  Addition / Alteration       2019    V6B 2L3   49.280142, -123.1095515
4  Addition / Alteration       2019        N/A  49.2634463, -123.1867632
5           New Building       2019    V5X 2Y4  49.2158549, -123.1154785
6           New Building       2019    V5R 3J7  49.2360209, -123.0558552


/tmp/ipykernel_39380/2828145549.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Cordinates'] = filtered_df['geo_point_2d']


In [20]:
df.to_csv('building-permits-cleaned.csv', index=False)